# Data cleaning of NCDB dataset

In [2]:
import pandas as pd  
import numpy as np
import warnings 
import math
warnings.simplefilter('ignore')

In [3]:
##  custom function!!!
def N_table(df, var):
    table =  (
        df
        .groupby(var, dropna = False)
        .size()
        .to_frame('N')
        .reset_index()
    )
    return table

In [4]:
# first dataset: use for training 
NCDB = pd.read_csv("../data/raw/Merkel.csv")
NCDB.shape

(34886, 126)

In [5]:
NCDB.columns

Index(['PUF_CASE_ID', 'PUF_FACILITY_ID', 'FACILITY_TYPE_CD',
       'FACILITY_LOCATION_CD', 'AGE', 'SEX', 'RACE', 'SPANISH_HISPANIC_ORIGIN',
       'INSURANCE_STATUS', 'MED_INC_QUAR_00',
       ...
       'RX_HOSP_SURG_PRIM_SITE', 'RX_HOSP_CHEMO', 'RX_HOSP_IMMUNOTHERAPY',
       'RX_HOSP_HORMONE', 'RX_HOSP_OTHER', 'PUF_MULT_SOURCE',
       'PUF_REFERENCE_DATE_FLAG', 'RX_SUMM_SCOPE_REG_LN_2012',
       'RX_HOSP_DXSTG_PROC', 'PALLIATIVE_CARE_HOSP'],
      dtype='object', length=126)

### Filter for Merkel cell carcinoma 
Histology code 8247

In [6]:
NCDB = NCDB[NCDB.HISTOLOGY == 8247]
NCDB.shape

(14948, 126)

In [7]:
# PUF_CASE_ID = id number
# PUF FACIILITY ID = de-identified cancer site.
# FACILITY TYPE = type of location ... not avail for everyone
NCDB.head()

,PUF_CASE_ID,PUF_FACILITY_ID,FACILITY_TYPE_CD,FACILITY_LOCATION_CD,AGE,SEX,RACE,SPANISH_HISPANIC_ORIGIN,INSURANCE_STATUS,MED_INC_QUAR_00,...,RX_HOSP_SURG_PRIM_SITE,RX_HOSP_CHEMO,RX_HOSP_IMMUNOTHERAPY,RX_HOSP_HORMONE,RX_HOSP_OTHER,PUF_MULT_SOURCE,PUF_REFERENCE_DATE_FLAG,RX_SUMM_SCOPE_REG_LN_2012,RX_HOSP_DXSTG_PROC,PALLIATIVE_CARE_HOSP
7,Ddd391a23-a24e-4cc2-bb8f-722d75cb44ab,AQGTAXQUGM,2.0,9.0,90,2,1,0,3,4.0,...,0,0,0,0,0,0,1,NaN,0,0
8,D5383f9e2-76b0-48f6-b820-1a375f274b36,LVMXOQOZBB,3.0,3.0,71,1,1,0,1,3.0,...,27,0,0,0,0,0,1,NaN,0,0
12,D49ae77af-8319-4805-a13c-b7070a1ba25d,BTFNKIQFXQ,3.0,4.0,87,1,1,0,3,NaN,...,20,0,0,0,0,1,1,NaN,0,0
14,Dfd47ceb1-8ecf-42ef-8b66-17c87587cc96,IOEYAMIRQV,3.0,6.0,43,1,1,0,1,4.0,...,0,0,0,0,0,1,1,2.0,0,0
18,D2dc0d6f0-2160-421b-8bdf-0fb260950e04,HIXOPCCPFV,2.0,9.0,81,1,1,0,1,4.0,...,0,0,0,0,0,0,1,NaN,0,0


## variables of interest

#### FEATURES
- Primary Site on body
- LYMPH_VASCULAR_INVASION 
- CS_Tumor_Size (width in mm)
- CS Site Specific Factor 1 (depth in mm)
- CS Site Specific Factor 20 (tumor-infiltrating lymphocytes)
- CS Site Specific Factor 22 (immuno)
- also include Age, Sex

#### OUTCOMES
- Regional Nodes Postive = number of positive lymph nodes
- CS Site specific Factor 3 = clinical status of lymph node mets


#### Recode PRIMARY_SITE to tumor_site

Primary site of lesion
 - Head and neck = C440, C441, C442, C444 
 - Trunk = C445 
 -  Extremity = C446, C447
 - Other = Everything else
 -  datatype: categorical


In [8]:
# recode variable: primary site 
def recode_site(var):
    if (var == 'C440') | (var == 'C441') | (var == 'C442') | (var == 'C444'):
        recode_var = 'head_neck'
    elif (var == 'C445'):
        recode_var = 'trunk'
    elif (var == 'C446') | (var == 'C447'):
        recode_var = 'extremity'
    else:
        recode_var = 'other'
    return recode_var

# apply rercoding  and convert to categorical variable
NCDB['tumor_site'] = NCDB.PRIMARY_SITE.map(recode_site).astype('category')
N_table(NCDB, 'tumor_site')

,tumor_site,N
0,extremity,5917
1,head_neck,2440
2,other,5072
3,trunk,1519


#### Recode LYMPH_VASCULAR_INVASION to lymph_vasc_invasion

0=No, 1= Yes, 8 or 9 =no information, recode to NA

In [9]:
# recode variable: mark 8 and 9 as None / Null / NaN
NCDB['lymph_vasc_invasion'] = NCDB.LYMPH_VASCULAR_INVASION.replace([8,9], np.nan)
N_table(NCDB, 'lymph_vasc_invasion')
# *** PROBLEM: ALOT OF MISSING DATA !

,lymph_vasc_invasion,N
0,0.0,3139
1,1.0,1783
2,NaN,10026


#### Recode CS_SITESPECIFIC_FACTOR_1 to tumor_depth

Measured thickness (depth) of tumor, continuous

- 0 = no mass found... recode to NA, or leave as zero?
- 00.1-979 = 0.1 mm - 97.9 mm measurements 
- 98.0 = 98.0mm or larger 
- 98.8/99.0/99.8/99.9 = Not applicable... recode to NA 
- infering that 98.7 and 88.8 are also  Not applicable, recoded to NA.

 see http://web2.facs.org/cstage0205/merkelcellskin/MerkelCellSkin_jpt.html)
 

In [10]:
# replace invalid with NA
NCDB['tumor_depth'] = NCDB.CS_SITESPECIFIC_FACTOR_1.\
replace([988.0, 990.0,998.0, 999.0, 888, 987], np.nan)# becomes NaN

# convert units to mm
NCDB['tumor_depth'] = NCDB.tumor_depth/10

# compute valid data metrics 
valid = (NCDB.tumor_depth > 0) & (NCDB.tumor_depth <= 98.0)
print("Tumor depth provided: {}".format(sum(valid)))
print("No mass found (Zero): {}".format(sum(NCDB.tumor_depth == 0)))
print("Not valid data (NaN): {}".format(NCDB.tumor_depth.isnull().sum()))

NCDB.tumor_depth.describe()

Tumor depth provided: 3460
No mass found (Zero): 253
Not valid data (NaN): 11235


count    3713.000000
mean        7.990250
std        14.397075
min         0.000000
25%         1.000000
50%         3.000000
75%         9.000000
max        98.000000
Name: tumor_depth, dtype: float64

#### Recoding CS_SITESPECIFIC_FACTOR_20 to tumor_lymphocytes

Tumor infiltrating lymphocytes, categorical 
- 000 = negative
- 010 = weakly present
- 020 = strongly present
- 030 = present -  but no information on how strong
- 988, 998, 999 - Not available, recode to NA

recding into binary >0 positive 

http://web2.facs.org/cstage0205/merkelcellskin/MerkelCellSkin_sra.html


In [11]:
# replace 988 and 999 and 998 with NAs and convert 
NCDB['tumor_lymphocytes'] = NCDB.CS_SITESPECIFIC_FACTOR_20.\
replace([998.0, 988.0, 999.0], np.nan).\
apply(lambda x: x if math.isnan(x) else x>0).\
replace({True: 1, False: 0}).astype('Int32')

N_table(NCDB, 'tumor_lymphocytes')
#** CANT GET RID OF 988

,tumor_lymphocytes,N
0,0,1875
1,1,813
2,<NA>,12260


#### Recoding CS_SITESPECIFIC_FACTOR_22 to immuno_suppressed

Immunosuppression, binarize
- 000 = negative
- 988 or 999 = no information, recode to NA
- anything else = positive

http://web2.facs.org/cstage0205/merkelcellskin/MerkelCellSkin_src.html

In [12]:
# replace 988 and 999 and 998 with NAs 
NCDB['CS_SITESPECIFIC_FACTOR_22_R'] = NCDB.CS_SITESPECIFIC_FACTOR_22.\
replace([998, 988, 999.0],  np.nan)
#** CANT GET RID OF 988

In [13]:
# make binary whether positive or negative 
NCDB['immuno_suppressed'] = NCDB.CS_SITESPECIFIC_FACTOR_22_R.\
apply(lambda x: x if math.isnan(x) else x>0).\
replace({True: 1, False: 0}).astype('Int32')
N_table(NCDB, 'immuno_suppressed')

,immuno_suppressed,N
0,0,4113
1,1,524
2,<NA>,10311


#### RECODING TUMOR_SIZE to tumor_size_bins_cm

Tumor size, but this changed in newest data version...
- 000 = no mass found 
- 001-988 = exact size in mm ... but move decimal.
- 989 = 989 mm or larger
- 990  = foci only, no size given
- 991 = less than 1 cm (10 mm)
- 992 = between 1-2 cm 
- 993 = between 2-3 cm 
- 994 = between 3-4 cm
- 995 = between 4-5 cm 
- 996 = greater than 5cm (50 mm)
- 999 = No info available 

Recode to bins of 10 mm, 1 cm 
- 0 (no mass) 
- 991 or < 10mm = less than 1 cm
- 992 or < 20mm = <2 cm
- 993 or < 30 mm = <3 cm
- 994 or < 40 mm = <4 cm
- 995 or < 50 mm = <5 cm
- 996 or > 50 mm or  989 = > 5 cm 
- 990 and 999 = NaN


see http://web2.facs.org/cstage0205/merkelcellskin/MerkelCellSkin_apo.html)

In [14]:
# recode variable: primary size, in cm
def recode_size(var):
    if (var == 0):
        recode_var = 0
    elif (var == 991) | (var < 100):
        recode_var = 1
    elif (var == 992) | (var < 200):
        recode_var = 2
    elif (var == 993) | (var < 300):
        recode_var = 3
    elif (var == 994)  | (var < 400):
        recode_var = 4
    elif (var == 995) | (var < 500):
        recode_var = 5
    elif(var == 996) | (var < 990):
        recode_var = 6 # but this really means 6 +
    else:
        recode_var = None
    return recode_var

# apply rercoding  and convert to categorical variable
NCDB['tumor_size_bins_cm'] = NCDB['TUMOR_SIZE'].map(recode_size)
NCDB['tumor_size_bins_cm'].value_counts(dropna=False)

1.0    10034
NaN     3521
2.0      629
0.0      520
5.0       99
3.0       91
4.0       28
6.0       26
Name: tumor_size_bins_cm, dtype: int64

#### Demographic variables

In [15]:
# all plausible values.
NCDB.AGE.describe()

count    14948.000000
mean        74.221702
std         10.998389
min         21.000000
25%         67.000000
50%         76.000000
75%         83.000000
max         90.000000
Name: AGE, dtype: float64

In [16]:
# 1 = male, 2 = female 
N_table(NCDB, 'SEX')

,SEX,N
0,1,9431
1,2,5517



## Outcome variables of interest
Metastasis of cancer - positive reading in lymph node biopsy

Based on two variables:
- Regional Nodes Postive = number of positive lymph nodes
- CS_SITESPECIFIC_FACTOR_3 = clinical status of lymph node mets
http://web2.facs.org/cstage0205/merkelcellskin/MerkelCellSkin_lpq.html

#### Recoding REGIONAL_NODES_POSITIVE to regional_nodes_positive_bin

- 00 = all nodes are negative, recode to positive
- 01-89 =  exact # of positive nodes, recode to postive
- 90 = more than 90 nodes are positive, recode to positive
- 95 = positive aspiration of lymph nodes was performed, recode to positive
- 97 = positive nodes, but # unspecified, recode to positive
- 98 = no nodes were examined, recoded to NaN
- 99 = not applicable, recode to NaN

Recode to binary variable, where 1 = positive regional nodes, 9 = negative regional nodes

In [17]:
# raw variable = number of positive lymph nodes detected in surgery 
NCDB.REGIONAL_NODES_POSITIVE.describe()

count    14948.000000
mean        41.748261
std         47.684039
min          0.000000
25%          0.000000
50%          2.000000
75%         98.000000
max         99.000000
Name: REGIONAL_NODES_POSITIVE, dtype: float64

In [33]:
# make a binary variable.
NCDB['regional_nodes_positive_bin'] = NCDB.REGIONAL_NODES_POSITIVE.\
replace([99, 98], np.nan)\
apply(lambda x: x if math.isnan(x) else x>0).\
replace({True: 1, False: 0}).

# ** MISSING A LOT OF DATA 
N_table(NCDB, 'regional_nodes_positive_bin')

,regional_nodes_positive_bin,N
0,0.0,4862
1,1.0,4129
2,NaN,5957


#### Recode CS_SITESPECIFIC_FACTOR_3 to lymph_node_mets_bin

- 0 = no mets, negative
-  5 = clinically negative mets, recode as negative
- 10 = micro-mets, positive cases from biopsy
- 20 = macro-mets, exclude from consideration (NA) because positive case without biospsy needed
- 100 = spreading mets, exclude from consideration (NA) because positive case without biospsy needed
- 150 = spreadingg more , exclude from consideration (NA)because positive case without biospsy needed
- 888, 988, 999 = not applicable, convert to NA


http://web2.facs.org/cstage0205/merkelcellskin/MerkelCellSkin_lpq.html


In [19]:
# raw variable = 
NCDB.CS_SITESPECIFIC_FACTOR_3.value_counts(dropna = False)

988.0    5385
5.0      4027
20.0     1828
0.0      1258
999.0    1141
10.0      851
888.0     302
150.0      92
100.0      49
998.0      15
Name: CS_SITESPECIFIC_FACTOR_3, dtype: int64

In [59]:
# make a binary variable, metastasis present or absent.
NCDB['lymph_node_mets_bin'] = NCDB.CS_SITESPECIFIC_FACTOR_3.\
replace([888, 988, 999, 998, 997, 20,  100, 150], np.nan).\
apply(lambda x: x if math.isnan(x) else (x > 5)).\
replace({True: 1, False: 0})

N_table(NCDB, 'lymph_node_mets_bin')

,lymph_node_mets_bin,N
0,0.0,5285
1,1.0,851
2,NaN,8812


## Coding primary outcome of positive_metastasis:


In [62]:
# Looks like we need a tie breaker! 
N_table(NCDB, ['lymph_node_mets_bin',  'regional_nodes_positive_bin'])


,lymph_node_mets_bin,regional_nodes_positive_bin,N
0,0.0,0.0,2874
1,0.0,1.0,11
2,0.0,NaN,2400
3,1.0,0.0,20
4,1.0,1.0,815
5,1.0,NaN,16
6,NaN,0.0,1968
7,NaN,1.0,3303
8,NaN,NaN,3541


In [63]:
def recode_outcome (df):
    # if they have both variables... (if both are NOT na) 
    if (not math.isnan(df.regional_nodes_positive_bin)) & (not math.isnan(df.lymph_node_mets_bin)):
        # provide value if they both agree
        if (df.regional_nodes_positive_bin == 1) & (df.lymph_node_mets_bin == 1): 
            return 1
        elif (df.regional_nodes_positive_bin == 0) & (df.lymph_node_mets_bin == 0): 
            return 0
        # if they don't agree, exclude 
        else: 
            return np.nan 
    # if they only have one variable, use that
    elif (not math.isnan(df.regional_nodes_positive_bin)):
        return df.regional_nodes_positive_bin
    elif (not math.isnan(df.lymph_node_mets_bin)):
        return df.lymph_node_mets_bin
        
 
# apply recoding to datafrarme.
NCDB['metastasis'] = NCDB.apply(recode_outcome, axis = 1)
N_table(NCDB, 'metastasis')



,metastasis,N
0,0.0,7242
1,1.0,4134
2,NaN,3572


## Make NCDB dataset with full feature set

In [68]:
# select features and outcome variable
NCDB1 = NCDB[['AGE', 'SEX', 'tumor_size_bins_cm', 'tumor_site', 
                    'immuno_suppressed', 'tumor_lymphocytes',
                     'lymph_vasc_invasion', 'tumor_depth',
              'metastasis']]
NCDB1.shape

(14948, 9)

In [69]:
# drop any rows with missing data.
print("Missingness of features:")
missing_table  = ( 
    NCDB1.isna()
    .sum()
    .to_frame("Missing")
    .reset_index()
)
missing_table

Missingness of features:


,index,Missing
0,AGE,0
1,SEX,0
2,tumor_size_bins_cm,3521
3,tumor_site,0
4,immuno_suppressed,10311
5,tumor_lymphocytes,12260
6,lymph_vasc_invasion,10026
7,tumor_depth,11235
8,metastasis,3572


In [70]:
NCDB1_cleaned = NCDB1.dropna(axis = 'rows', how = 'any')

# count how many dropped 
original_rows = NCDB1.shape[0]
rows_remaining  = NCDB1_cleaned.shape[0]
dropped_rows = (original_rows - rows_remaining)
prop_missing = round(dropped_rows / original_rows, 2)

NCDB1_cleaned.to_csv("../data/cleaned/NCDB_cleaned_N924.csv", index = False)

print("Rows with full data: {}".format(rows_remaining))
print("Rows dropped for missingness: {}".format(dropped_rows))
print("Proprtion of rows with missing data: {}".format(prop_missing))


Rows with full data: 924
Rows dropped for missingness: 14024
Proprtion of rows with missing data: 0.94


### Make NCDB dataset to merge with SEER

Variables not available in SEER:
- lymph_vasc_invasion
- immuno_suppressed
- tumor_lymphocytes
- tumor depth

In [77]:
# select features and outcome variable
NCDB2 = NCDB[['AGE', 'SEX', 'tumor_size_bins_cm', 'tumor_site',
              'metastasis']]
NCDB2.shape

(14948, 5)

In [78]:
# drop any rows with missing data.
print("Missingness of features:")
missing_table  = ( 
    NCDB2.isna()
    .sum()
    .to_frame("Missing")
    .reset_index()
)
missing_table

Missingness of features:


,index,Missing
0,AGE,0
1,SEX,0
2,tumor_size_bins_cm,3521
3,tumor_site,0
4,metastasis,3572


In [79]:
NCDB2_cleaned = NCDB2.dropna(axis = 'rows', how = 'any')

# count how many dropped 
original_rows = NCDB2.shape[0]
rows_remaining  = NCDB2_cleaned.shape[0]
dropped_rows = (original_rows - rows_remaining)
prop_missing = round(dropped_rows / original_rows, 2)

NCDB2_cleaned.to_csv("../data/cleaned/NCDB_cleaned_for_SEER_merge.csv", index = False)

print("Rows with full data: {}".format(rows_remaining))
print("Rows dropped for missingness: {}".format(dropped_rows))
print("Proprtion of rows with missing data: {}".format(prop_missing))


Rows with full data: 9109
Rows dropped for missingness: 5839
Proprtion of rows with missing data: 0.39


In [80]:
NCDB2_cleaned.metastasis.value_counts(dropna= False)

0.0    5887
1.0    3222
Name: metastasis, dtype: int64